<a href="https://colab.research.google.com/github/5ugika2u/my-first-blog/blob/master/%E7%99%BA%E5%B1%95%E7%A0%94%E7%A9%B6%E5%89%AF%E3%83%86%E3%83%BC%E3%83%9E%E7%A0%94%E7%A9%B6Sentence_BERT%E3%82%92%E7%94%A8%E3%81%84%E3%81%9F%E6%84%8F%E8%A6%8B%E5%88%86%E3%81%AE%E5%88%86%E9%A1%9E_ipynb_%E3%81%AE%E3%82%B3%E3%83%94%E3%83%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **副テーマ研究課題Sentence BERTを用いた意見分の分類**

意見文、非意見文のラベルが付与された楽天のデータセットを使用し、意見文か、非意見文かを判定するモデルを構築する。

**モデルの構築に必要なライブラリをインストールする。**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd "/content/drive/MyDrive/副テーマ研究/ShiraiLab_minorR_data"

/content/drive/MyDrive/副テーマ研究/ShiraiLab_minorR_data


In [ ]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 447 kB 4.6 MB/s 
     |████████████████████████████████| 574 kB 11.0 MB/s 
     |████████████████████████████████| 880 kB 20.5 MB/s 
     |████████████████████████████████| 132 kB 25.4 MB/s 
     |████████████████████████████████| 1.2 MB 28.3 MB/s 
     |████████████████████████████████| 79 kB 8.6 MB/s 
     |████████████████████████████████| 8.9 MB 35.8 MB/s 
     |████████████████████████████████| 138 kB 47.9 MB/s 
     |████████████████████████████████| 127 kB 48.9 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=5228e32584b03f475fd3076a45c1e95564e4950a042a2bf598a850649cb9f40b
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    U

In [ ]:
# ライブラリをインポートする。
import numpy as np
import pandas as pd
import scipy as sp
from scipy import stats
import nltk as nl

In [ ]:
!pip install transformers==2.3.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install mecab-python3
!pip install unidic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for unidic: filename=unidic-1.1.0-py3-none-any.whl size=7426 sha256=3d54859e1855707c633587f246564d431b5aaea7e8bbce6e4b8a4518928d1758
  Stored in directory: /root/.cache/pip/wheels/ce/4d/f1/170bb74b559ca338113c0315c9805e16dfd0a12411ec6b1122
Successfully built unidic


**ディクショナリをダウンロードする**

In [ ]:
!python -m unidic download
import unidic

download url: https://cotonoha-dic.s3-ap-northeast-1.amazonaws.com/unidic-3.1.0.zip
Dictionary version: 3.1.0+2021-08-31
unidic-3.1.0.zip: 100% 526M/526M [00:23<00:00, 22.3MB/s]
Finished download.
Downloaded UniDic v3.1.0+2021-08-31 to /usr/local/lib/python3.7/dist-packages/unidic/dicdir


**学習に用いるデータセットを読み込む。Grabツールを使用し、任意のファイルを取得する**

In [ ]:
from glob import glob

In [ ]:
filepaths =glob('*.txt')
filepaths

['data_test.txt',
 'data_train.txt',
 'requirements.txt',
 'Instruction.txt',
 'data_test3.txt',
 'data_train3.txt']

**取得したデータセットをPandasのデータフレームとして読み込む。**

In [ ]:
df1 = pd.read_table("data_train3.txt",header=None)
df1

,0,1,2
0,1,no,数年前、友人が宿泊しているところへ招待されて、次に博多で泊まるならココ！と決めていました。
1,2,yes,部屋もベッドも広く、また、チェックインの時のプレゼントが選べるのもちょっとうれしかったです。
2,3,yes,難を言えば湯船が狭いことと、喫煙室であったため、白い壁がヤニで茶色く変色していたこと。
3,4,yes,特にソファーに座るとちょうど目線がそこに行くためかなり気になりました。
4,5,yes,また、空気清浄機がかなりうるさいです。
...,...,...,...
3671,4072,yes,チェックインの方法は画期的です。
3672,4073,yes,おかげで自分の部屋のような感じで使えます。
3673,4074,yes,お気に入りになりました。
3674,4075,yes,前にも書きましたが、夕食を食べに出掛けるには少し繁華街から離れていますが、私にとっては、それ...


In [ ]:
df2 = pd.read_table("data_test3.txt",header=None)
df2

,0,1,2
0,14,yes,立地が競合他ホテルに比較してやや不利かも知れませんが、それをカバーする工夫や努力が感じられ好...
1,43,yes,シャワーしか使用しない私でも狭くて難儀したくらいです。
2,44,yes,立地、金額、ベッドの大きさ、部屋の清潔感、従業員の応対、全て今イチバンお気に入りのビジネスホ...
3,64,yes,現在釧路で一番良いホテルではないでしょうか。
4,82,yes,エコを考えての事だと思いますが‥お部屋の１つの他に、フロント脇に置かれているアメニティーグッ...
...,...,...,...
395,3975,yes,クチコミ評判どおり食事が美味しく、ボリューム満点でした。
396,3978,yes,子供たちが遊べるスペースもあり、大変満足です。
397,3995,yes,従業員の接客がもの凄く良かった。
398,4011,no,先日は、お世話になりました。


**データに対して、ラベルを目的変数。文章を説明変数としてスプリットする。テストデータ、トレインデータそれぞれに行う**

In [ ]:
# 説明変数、もとのデータから説明変数を抽出
explain1 = df1.iloc[:, 2]
# 目的変数、もとのデータセットの目的変数を抽出
target1 = df1.iloc[:, 1]

In [ ]:
# 説明変数、もとのデータから説明変数を抽出
explain2 = df2.iloc[:, 2]
# 目的変数、もとのデータセットの目的変数を抽出
target2 = df2.iloc[:, 1]

In [ ]:
explain1


0           数年前、友人が宿泊しているところへ招待されて、次に博多で泊まるならココ！と決めていました。
1          部屋もベッドも広く、また、チェックインの時のプレゼントが選べるのもちょっとうれしかったです。
2             難を言えば湯船が狭いことと、喫煙室であったため、白い壁がヤニで茶色く変色していたこと。
3                     特にソファーに座るとちょうど目線がそこに行くためかなり気になりました。
4                                     また、空気清浄機がかなりうるさいです。
                              ...                        
3671                                     チェックインの方法は画期的です。
3672                                おかげで自分の部屋のような感じで使えます。
3673                                         お気に入りになりました。
3674    前にも書きましたが、夕食を食べに出掛けるには少し繁華街から離れていますが、私にとっては、それ...
3675                           今の時期なら寒い中歩いていって、熱燗が美味いとか＾＾
Name: 2, Length: 3676, dtype: object

In [ ]:
explain2

0      立地が競合他ホテルに比較してやや不利かも知れませんが、それをカバーする工夫や努力が感じられ好...
1                            シャワーしか使用しない私でも狭くて難儀したくらいです。
2      立地、金額、ベッドの大きさ、部屋の清潔感、従業員の応対、全て今イチバンお気に入りのビジネスホ...
3                                 現在釧路で一番良いホテルではないでしょうか。
4      エコを考えての事だと思いますが‥お部屋の１つの他に、フロント脇に置かれているアメニティーグッ...
                             ...                        
395                         クチコミ評判どおり食事が美味しく、ボリューム満点でした。
396                              子供たちが遊べるスペースもあり、大変満足です。
397                                     従業員の接客がもの凄く良かった。
398                                       先日は、お世話になりました。
399                           灯明祭りにも行きましたがとても綺麗で良かったですね。
Name: 2, Length: 400, dtype: object

**Bag of words モデルの構築に必要な辞書作成のために、テストデータとトレインデータを結合する。**

In [ ]:
explain3 = pd.concat([explain1, explain2])
explain3

0       数年前、友人が宿泊しているところへ招待されて、次に博多で泊まるならココ！と決めていました。
1      部屋もベッドも広く、また、チェックインの時のプレゼントが選べるのもちょっとうれしかったです。
2         難を言えば湯船が狭いことと、喫煙室であったため、白い壁がヤニで茶色く変色していたこと。
3                 特にソファーに座るとちょうど目線がそこに行くためかなり気になりました。
4                                 また、空気清浄機がかなりうるさいです。
                            ...                      
395                      クチコミ評判どおり食事が美味しく、ボリューム満点でした。
396                           子供たちが遊べるスペースもあり、大変満足です。
397                                  従業員の接客がもの凄く良かった。
398                                    先日は、お世話になりました。
399                        灯明祭りにも行きましたがとても綺麗で良かったですね。
Name: 2, Length: 4076, dtype: object

In [ ]:
pip install neologdn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 57 kB 2.4 MB/s 
  Created wheel for neologdn: filename=neologdn-0.5.1-cp37-cp37m-linux_x86_64.whl size=172960 sha256=e6e6d15223cbe884d4597904c3eb4d7d7c7ab4cfd268747f0a660b7439b35020
  Stored in directory: /root/.cache/pip/wheels/19/15/5c/55b33d02e16129ef81313e4c86e473d6dd1cecf7317a525a9b
Successfully built neologdn


## **Bag of words**

  **入力の文章に対し、自立語抽出を行う。このとき、原型の抽出を行う。**

In [ ]:
# 重複しますが、念のためもう一度インポート
import MeCab
import re
# import neologdn
import neologdn

import re


mecab = MeCab.Tagger('')

def wakati(sentences):
    
    sentences_word_list = []

    for sentence in sentences:
        
        one_sentence_word_str = ''
        node = mecab.parse('')	
        
        sentence1 = neologdn.normalize(sentence)
        tmp = re.sub(r'(\d)([,.])(\d+)', r'\1\3', sentence1)
        text_replaced_number = re.sub(r'\d+', ' ', tmp)
        tagger = MeCab.Tagger()
        node =  tagger.parseToNode(text_replaced_number)
        
        while node:
            features = node.feature.split(',')
            if features[0] != 'BOS/EOS': 
                if node.feature.split(",")[0] == "名詞":
                    word = node.surface
                    one_sentence_word_str = one_sentence_word_str + " " + word
                elif node.feature.split(",")[0] =="動詞":
                    word = node.feature.split(",")[7] 
                    one_sentence_word_str = one_sentence_word_str + " " + word
                elif node.feature.split(",")[0] =="形容詞":
                    word = node.feature.split(",")[7] 
                    one_sentence_word_str = one_sentence_word_str + " " + word
                elif node.feature.split(",")[0] =="副詞":
                    word = node.feature.split(",")[7] 
                    one_sentence_word_str = one_sentence_word_str + " " + word
        
            node = node.next
        sentences_word_list.append(one_sentence_word_str)
    return sentences_word_list

wakati1 = wakati(explain1)
wakati2 = wakati(explain2)
wakati3 = wakati(explain3)


In [ ]:
wakati_pd1 = pd.DataFrame(wakati1)
print(wakati_pd1)

                                                    0
0      数 年 前 友人 宿泊 為る 居る ところ 招待 為る 次 博多 泊まる ココ 決める 居る
1                 部屋 ベッド 広い チェック イン 時 プレゼント 選ぶ 一寸 嬉しい
2      難 言う 湯船 狭い こと 喫煙 有る ため 白い 壁 ヤニ 茶色い 変色 為る 居る こと
3                      特に ソファー 座る 丁度 目線 行く ため 可成 気 成る
4                                       空気 清浄 機 可成 煩い
...                                               ...
3671                                    チェック イン 方法 画期
3672                                  おかげ 自分 部屋 感じ 使う
3673                                          気 入る 成る
3674       前 書く 夕食 食べる 出掛ける 少し 離れる 居る 取る 程良い 距離 成る 居る
3675                            今 時期 寒い 中 歩く 行く 熱燗 旨い

[3676 rows x 1 columns]


 **Bag of words を用いて、ベクトル化を行う。**

**辞書を定義するため、vectorizer.fit_transformを用いて、辞書の作成を行う。このとき、辞書作成用に予め用意したwakati3を用いる。**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(token_pattern=u'(?u)\\b\\w+\\b')
dic = vectorizer.fit_transform(wakati3)

In [ ]:
print(vectorizer.get_feature_names())

['am', 'apa', 'b', 'bar', 'bgm', 'c', 'cd', 'concert', 'conserttour', 'cp', 'dvd', 'ev', 'f', 'front', 'get', 'good', 'gp', 'h', 'hp', 'i', 'in', 'it', 'jal', 'jr', 'jtb', 'k', 'kinko', 'km', 'lan', 'm', 'museum', 'n', 'no', 'o', 'oap', 'occupancytax', 'off', 'ok', 'on', 'out', 'pc', 'pola', 'pr', 'ps', 'quo', 'r', 'roomcharge', 's', 't', 'tax', 'tbs', 'tdl', 'tdr', 'tel', 'tour', 'tv', 'tx', 'ub', 'usj', 'v', 'view', 'vod', 'w', 'wd', 'wifi', 'wtcc', 'www', 'あきら', 'あたり', 'あっさり', 'あて', 'あと', 'あめ', 'あり', 'ありあり', 'あれこれ', 'いかん', 'いく', 'いくら', 'いっぱい', 'いま', 'いまどき', 'いらっしゃる', 'うえ', 'うずしお', 'うち', 'うっかり', 'うつりかわり', 'うどん', 'うろうろ', 'うわさ', 'うんざり', 'えび', 'おかげ', 'おかず', 'おざなり', 'おしぼり', 'おしゃれ', 'おじ', 'おなか', 'おにぎり', 'おば', 'おまけ', 'おやつ', 'おろし蕎麦', 'おんど', 'お洒落', 'お腹', 'お袋', 'お陰', 'かけ流し', 'かげん', 'かすが', 'かた', 'かちあう', 'かに', 'かばん', 'かみそり', 'かわり', 'かんじ', 'かんぽ', 'がく', 'がしゃがしゃ', 'がたごと', 'がちゃがちゃ', 'がっかり', 'がっつり', 'がらがら', 'き', 'きちんと', 'きつい', 'きな粉', 'きり', 'ぎりぎり', 'く', 'くじ', 'くじ引き', 'くすむ', 'くっ付く', 'ぐっすり', 'ぐら', 'ぐらぐ

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
print(vectorizer.vocabulary_)

{'数': 2488, '年': 2158, '前': 1565, '友人': 1655, '宿泊': 2040, '為る': 2986, '居る': 2096, 'ところ': 232, '招待': 2406, '次': 2739, '博多': 1636, '泊まる': 2844, 'ココ': 531, '決める': 2827, '部屋': 3839, 'ベッド': 907, '広い': 2177, 'チェック': 711, 'イン': 402, '時': 2573, 'プレゼント': 894, '選ぶ': 3829, '一寸': 1081, '嬉しい': 1980, '難': 3960, '言う': 3579, '湯船': 2938, '狭い': 3043, 'こと': 147, '喫煙': 1781, '有る': 2622, 'ため': 204, '白い': 3122, '壁': 1861, 'ヤニ': 993, '茶色い': 3461, '変色': 1875, '特に': 3029, 'ソファー': 683, '座る': 2192, '丁度': 1104, '目線': 3140, '行く': 3497, '可成': 1689, '気': 2790, '成る': 2356, '空気': 3251, '清浄': 2908, '機': 2729, '煩い': 3009, '洗練': 2860, '雰囲気': 3972, '中々': 1170, 'もの': 331, '月': 2616, '日': 2534, '泊': 2842, '温泉': 2930, '食事': 4037, '限る': 3933, '価格': 1362, '以上': 1283, '思う': 2296, '古い': 1680, 'そこそこ': 190, 'リニューアル': 1026, '程度': 3234, '掃除': 2426, '一方': 1086, 'アメニティー': 381, '貧弱': 3649, '普通': 2585, '旅館': 2530, '無い': 2988, '持参': 2415, '必要': 2273, '今回': 1243, 'ゆっくり': 341, '旨い': 2548, '魚': 4109, '食う': 4033, 'テーマ': 737, '満足': 2949, '仲居'

In [ ]:
print(dic)

  (0, 2488)	1
  (0, 2158)	1
  (0, 1565)	1
  (0, 1655)	1
  (0, 2040)	1
  (0, 2986)	2
  (0, 2096)	2
  (0, 232)	1
  (0, 2406)	1
  (0, 2739)	1
  (0, 1636)	1
  (0, 2844)	1
  (0, 531)	1
  (0, 2827)	1
  (1, 3839)	1
  (1, 907)	1
  (1, 2177)	1
  (1, 711)	1
  (1, 402)	1
  (1, 2573)	1
  (1, 894)	1
  (1, 3829)	1
  (1, 1081)	1
  (1, 1980)	1
  (2, 2986)	1
  :	:
  (4071, 4037)	1
  (4071, 3368)	1
  (4071, 240)	1
  (4071, 501)	1
  (4071, 932)	1
  (4071, 2947)	1
  (4071, 3605)	1
  (4072, 2622)	1
  (4072, 2949)	1
  (4072, 1918)	1
  (4072, 658)	1
  (4072, 1983)	1
  (4072, 3792)	1
  (4073, 3434)	1
  (4073, 2435)	1
  (4073, 2250)	1
  (4073, 3026)	1
  (4074, 2356)	1
  (4074, 1159)	1
  (4074, 1424)	1
  (4075, 3497)	1
  (4075, 3748)	1
  (4075, 3434)	1
  (4075, 2977)	1
  (4075, 3223)	1


In [ ]:
print(dic.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
vec_1_sub = vectorizer.transform(wakati1)

In [ ]:
vec_2_sub = vectorizer.transform(wakati2)

In [ ]:
vec_1 = vec_1_sub.toarray()
vec_1

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
vec_2 = vec_2_sub.toarray()
vec_2

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
vec1_pd1 = pd.DataFrame(vec_1)
vec1_pd1

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,4089,4090,4091,4092,4093,4094,4095,4096,4097,4098,4099,4100,4101,4102,4103,4104,4105,4106,4107,4108,4109,4110,4111,4112,4113,4114,4115,4116,4117,4118,4119,4120,4121,4122,4123,4124,4125,4126,4127,4128
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3671,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3672,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3673,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3674,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
vec2_pd1 = pd.DataFrame(vec_2)
vec2_pd1

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,4089,4090,4091,4092,4093,4094,4095,4096,4097,4098,4099,4100,4101,4102,4103,4104,4105,4106,4107,4108,4109,4110,4111,4112,4113,4114,4115,4116,4117,4118,4119,4120,4121,4122,4123,4124,4125,4126,4127,4128
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
396,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
397,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
398,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
target1

0        no
1       yes
2       yes
3       yes
4       yes
       ... 
3671    yes
3672    yes
3673    yes
3674    yes
3675    yes
Name: 1, Length: 3676, dtype: object

**SVMによる分類（Bag of words）**

In [ ]:
# SVMの学習
from sklearn import svm

def svbag(carnel):
    # 結果を出力してくれるライブラリをインポート
    from sklearn.metrics import classification_report
    classifier = svm.SVC(kernel=carnel, C=1, gamma=0.1)
    classifier.fit(vec1_pd1,target1)

    # 予測
    predict_label1 = classifier.predict(vec2_pd1)
    test_label = target2
    target_names = ["yes", "no"]
    result = classification_report(test_label, predict_label1,target_names=target_names)

    return result,predict_label1
bagrbf = svbag("rbf")
bagline = svbag("linear")
bagpoly = svbag("poly")
bagsig = svbag("sigmoid")



**RBFカーネル関数を用いた結果（Bag of words）**

In [ ]:
print(bagrbf[0])

              precision    recall  f1-score   support

         yes       0.86      0.63      0.73        93
          no       0.90      0.97      0.93       307

    accuracy                           0.89       400
   macro avg       0.88      0.80      0.83       400
weighted avg       0.89      0.89      0.88       400



**線形カーネル関数を用いた結果（Bag of words）**

In [ ]:
print(bagline[0])

              precision    recall  f1-score   support

         yes       0.77      0.71      0.74        93
          no       0.91      0.93      0.92       307

    accuracy                           0.88       400
   macro avg       0.84      0.82      0.83       400
weighted avg       0.88      0.88      0.88       400



**多項式カーネル関数を用いた結果（Bag of words）**

In [ ]:
print(bagpoly[0])

              precision    recall  f1-score   support

         yes       0.68      0.20      0.31        93
          no       0.80      0.97      0.88       307

    accuracy                           0.79       400
   macro avg       0.74      0.59      0.60       400
weighted avg       0.77      0.79      0.75       400



**シグモイドカーネル関数を用いた結果（Bag of words）**

In [ ]:
print(bagsig[0])

              precision    recall  f1-score   support

         yes       0.70      0.65      0.67        93
          no       0.89      0.92      0.90       307

    accuracy                           0.85       400
   macro avg       0.80      0.78      0.79       400
weighted avg       0.85      0.85      0.85       400



In [ ]:
#!/usr/local/bin/python3
# テスト用サンプル
import sys
from sentence_transformers import SentenceTransformer

model_path = "/content/drive/MyDrive/副テーマ研究/ShiraiLab_minorR_data/model"

print("loading Sentence BERT model",file=sys.stderr,flush=True)
model = SentenceTransformer(model_path, show_progress_bar=False)

sentence_list = ['ミーティングの発表をします。','準備はできてます。']

sentence_vectors = model.encode(sentence_list)

print('# 1=sentence, 2=first ten values of vector, 3=vector size')
for i in range(0,len(sentence_list)):
    print('1: '+sentence_list[i])
    print('2: '+str(sentence_vectors[i][0:10]))
    print('3: '+str(len(sentence_vectors[i])))

loading Sentence BERT model


# 1=sentence, 2=first ten values of vector, 3=vector size
1: ミーティングの発表をします。
2: [-1.5775018e+00  1.1613996e+00 -5.0669622e-01 -2.6725058e-02
  3.4613946e-01  9.2589229e-02  1.8837030e+00 -2.8558412e-01
  8.0851257e-02  7.2293938e-04]
3: 768
1: 準備はできてます。
2: [-0.50869495  0.71494967 -1.0624052   0.7291435  -0.6984611   0.8621531
  1.0153402  -0.92056453 -0.09095055 -0.5511198 ]
3: 768


In [ ]:
#!/usr/local/bin/python3

import sys
from sentence_transformers import SentenceTransformer

model_path = "/content/drive/MyDrive/副テーマ研究/ShiraiLab_minorR_data/model"

print("loading Sentence BERT model",file=sys.stderr,flush=True)
model = SentenceTransformer(model_path, show_progress_bar=False)

sentence_list = explain1 

sentence_vectors = model.encode(sentence_list)

print('# 1=sentence, 2=first ten values of vector, 3=vector size')
for i in range(0,len(sentence_list)):
    print('1: '+sentence_list[i])
    print('2: '+str(sentence_vectors[i][0:10]))
    print('3: '+str(len(sentence_vectors[i])))

loading Sentence BERT model


ストリーミング出力は最後の 5000 行に切り捨てられました。
3: 768
1: 8月に帰省し今回だったので経済的にしんどい帰省でしたが安いホテルでしたので助かりました。
2: [-0.24149442  0.58664405 -0.8796691   0.07380804 -0.9957323   0.06174699
  0.5182024  -0.03867079 -0.29070288 -1.4620726 ]
3: 768
1: 9月に母の88歳のお祝いでまた帰省しますので夏でも安い料金での提供をお願いいたしますフロント、お掃除、食事のスタッフの方々は感じの良い方ばかりでした。
2: [-0.82382596  0.02312755 -0.85355645 -0.28293768  0.4007872  -0.09538092
  1.2905349  -0.11253491 -0.8553929  -0.78490055]
3: 768
1: ありがとうございました。
2: [-1.2149969  -1.181721   -1.2573392   0.7816442   0.3704121   1.229902
  1.6393665  -0.14472452 -0.01611352 -0.42188436]
3: 768
1: 12/29に宿泊しました。
2: [-1.1436092   0.17308097  0.13162766  0.33513692 -1.9633561   0.07654071
  1.5362927  -0.13131404 -0.67052466  0.49437886]
3: 768
1: こちらのクチコミを参考に選んだのですが正解でした。
2: [-1.2629479   0.30803156  0.01406229 -0.5284726  -0.8768529   0.97762316
  0.6243241  -0.5593803  -0.71088904  0.24467097]
3: 768
1: 子供連れでしたが広い部屋を用意して頂きのんびりできました。
2: [-0.7766634   0.11739812 -1.3932914  -0.73396444 -0.15896054  0.5139817
 

In [ ]:
sentence_list2 = explain2

sentence_vectors2 = model.encode(sentence_list2)

print('# 1=sentence, 2=first ten values of vector, 3=vector size')
for i in range(0,len(sentence_list2)):
    print('1: '+sentence_list2[i])
    print('2: '+str(sentence_vectors2[i][0:10]))
    print('3: '+str(len(sentence_vectors2[i])))

# 1=sentence, 2=first ten values of vector, 3=vector size
1: 立地が競合他ホテルに比較してやや不利かも知れませんが、それをカバーする工夫や努力が感じられ好感が持てます。
2: [-0.3482409   0.2423363  -1.2489362  -0.54079086 -1.0051001  -0.0496973
  0.9122568  -0.58143127  0.02158922 -0.3604564 ]
3: 768
1: シャワーしか使用しない私でも狭くて難儀したくらいです。
2: [ 0.7388403  -0.08360618 -0.169157    0.18971615 -0.66505444  1.1230379
  0.99376124 -0.03148507 -1.1112702  -0.458566  ]
3: 768
1: 立地、金額、ベッドの大きさ、部屋の清潔感、従業員の応対、全て今イチバンお気に入りのビジネスホテルです。
2: [-0.54355055  0.16073692 -1.5550308   0.31232786  0.6420338   0.09140866
  0.33487934 -0.25008357 -0.27710804  0.06320287]
3: 768
1: 現在釧路で一番良いホテルではないでしょうか。
2: [-0.38775823 -0.26182    -0.793002    0.23505823  0.47329444  0.33499107
  1.3697877   0.07337852  0.25031823 -0.19393158]
3: 768
1: エコを考えての事だと思いますが‥お部屋の１つの他に、フロント脇に置かれているアメニティーグッズ同様、お茶類も数種類を置いてはいかがでしょうか？
2: [ 0.14258188 -0.08291256 -1.8521742  -0.40949893 -0.45982167  0.64093566
  0.6937095  -0.472426   -0.8760444   0.268266  ]
3: 768
1: また梅田ーホテル無料シャトルバスも出ているので、便利でした。
2

In [ ]:
sentence_vectors1_pd1 = pd.DataFrame(sentence_vectors)
sentence_vectors2_pd2 =pd.DataFrame(sentence_vectors2)

In [ ]:
# SVMの学習
from sklearn import svm

def svbert(carnel):
    # 結果を出力してくれるライブラリをインポート
    from sklearn.metrics import classification_report
    classifier = svm.SVC(kernel=carnel, C=1, gamma=0.1)
    classifier.fit(sentence_vectors1_pd1,target1)

    # 予測
    predict_label2 = classifier.predict(sentence_vectors2_pd2)
    test_label = target2
    target_names = ["yes", "no"]
    result2 = classification_report(test_label, predict_label2,target_names=target_names)
    return result2,predict_label2

bertrbf = svbert("rbf")
bertline = svbert("linear")
bertpoly = svbert("poly")
bertsig = svbert("sigmoid")

**RBFカーネル関数を用いた結果（BERT）**

In [ ]:
print(bertrbf[0])

              precision    recall  f1-score   support

         yes       1.00      0.24      0.38        93
          no       0.81      1.00      0.90       307

    accuracy                           0.82       400
   macro avg       0.91      0.62      0.64       400
weighted avg       0.86      0.82      0.78       400



**線形カーネル関数を用いた結果（BERT）**

In [ ]:
print(bertline[0])

              precision    recall  f1-score   support

         yes       0.76      0.75      0.76        93
          no       0.93      0.93      0.93       307

    accuracy                           0.89       400
   macro avg       0.84      0.84      0.84       400
weighted avg       0.89      0.89      0.89       400



**多項式カーネル関数を用いた結果（BERT）**

In [ ]:
print(bertpoly[0])

              precision    recall  f1-score   support

         yes       0.91      0.73      0.81        93
          no       0.92      0.98      0.95       307

    accuracy                           0.92       400
   macro avg       0.91      0.85      0.88       400
weighted avg       0.92      0.92      0.92       400



**シグモイドカーネル関数を用いた結果（BERT）**

In [ ]:
print(bertsig[0])

              precision    recall  f1-score   support

         yes       0.19      0.19      0.19        93
          no       0.75      0.74      0.75       307

    accuracy                           0.61       400
   macro avg       0.47      0.47      0.47       400
weighted avg       0.62      0.61      0.62       400



**値の違うindexとcolumnを特定する**


BERTモデルに於ける、分類結果をそれぞれのカーネルに於いて抽出

In [ ]:
bertrbf_pd = pd.Series(bertrbf[1])

In [ ]:
bertline_pd = pd.Series(bertline[1])

In [ ]:
bertpoly_pd = pd.Series(bertpoly[1])

In [ ]:
bertsig_pd = pd.Series(bertsig[1])

Bag of Wordsモデルに於ける、分類結果をそれぞれのカーネルに於いて抽出

In [ ]:
bagrbf_pd = pd.Series(bagrbf[1])

In [ ]:
bagline_pd = pd.Series(bagline[1])

In [ ]:
bagpoly_pd = pd.Series(bagpoly[1])

In [ ]:
bagsig_pd = pd.Series(bagsig[1])

In [ ]:
type(target2)

pandas.core.series.Series

In [ ]:
target2

0      yes
1      yes
2      yes
3      yes
4      yes
      ... 
395    yes
396    yes
397    yes
398     no
399    yes
Name: 1, Length: 400, dtype: object

In [ ]:
# BERT rbfカーネルモデルに於ける、正誤判定
bertrbf_comp = bertrbf_pd.compare(target2)
bertrbf_comp

,self,other
7,yes,no
12,yes,no
18,yes,no
27,yes,no
29,yes,no
...,...,...
359,yes,no
363,yes,no
366,yes,no
368,yes,no


In [ ]:
# BERT 線形カーネルモデルに於ける、正誤判定
bertline_comp = bertline_pd.compare(target2)
bertline_comp

,self,other
17,no,yes
18,yes,no
21,no,yes
37,no,yes
45,yes,no
60,no,yes
68,no,yes
81,yes,no
90,yes,no
98,yes,no


In [ ]:
bertsig_comp = bertsig_pd.compare(target2)
bertsig_comp

,self,other
2,no,yes
5,no,yes
7,yes,no
15,yes,no
18,yes,no
...,...,...
394,yes,no
395,no,yes
396,no,yes
397,no,yes


In [ ]:
# BERT 多項式カーネルモデルに於ける、正誤判定
bertpoly_comp = bertpoly_pd.compare(target2)
bertpoly_comp

,self,other
18,yes,no
37,no,yes
45,yes,no
73,yes,no
81,yes,no
98,yes,no
136,yes,no
155,yes,no
161,no,yes
162,yes,no


# **グリッドサーチを行う準備を行う**

In [ ]:
sentence_vectors1_pd1

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767
0,-0.312238,0.330007,-0.922178,-0.590664,-0.311608,-0.787505,0.683535,-0.202147,0.043942,-0.362537,-0.683769,0.192867,-0.914427,-0.077644,0.461754,-1.037730,-0.616900,-0.382177,-0.681145,1.500364,0.376420,-1.179792,-0.130584,-0.077526,-0.055792,-0.076235,-0.486196,0.002762,0.698907,-0.720542,0.992124,0.272427,0.111486,0.398325,1.358358,-0.340021,1.087985,0.226664,1.092323,-0.383439,...,-0.268894,0.079059,0.251017,-0.021267,-1.082281,0.615868,0.727195,0.582233,0.012167,1.038870,0.884038,-0.583681,0.447573,1.117432,-0.022901,0.476338,-0.187507,0.037361,-0.253796,-0.174433,-0.541638,0.014918,-0.871772,0.010125,0.030003,0.223326,0.054722,-2.008408,0.146070,-0.047044,-0.486090,0.539662,-0.469392,0.533446,-0.384141,0.015135,-0.575573,0.565052,0.300476,0.291620
1,-1.030763,0.955347,-1.540997,-0.285051,0.020167,0.496035,0.772115,-0.659970,0.503083,-0.696374,-0.401826,0.463217,-0.612228,-0.020825,0.508383,0.560354,1.258363,0.022446,-0.400604,0.610968,1.215740,-0.426224,-0.260288,0.070989,-0.319815,-0.195705,0.518609,-0.924011,0.753476,-0.764982,0.396179,0.780698,-0.626966,0.474035,0.463661,-0.432569,0.695716,-0.041585,0.736530,-0.447876,...,0.480373,-0.492486,0.356457,-0.607399,-0.233802,0.376061,-0.138405,0.087108,0.244690,0.159243,1.018089,-0.340612,-0.430836,1.741965,-0.397238,-0.681870,-0.256416,0.069347,-0.323959,0.246101,-0.399242,0.411663,-0.031079,-0.117548,-0.035029,0.567961,-0.656368,-0.710472,-0.501150,0.280536,-0.197805,-0.215502,1.347626,-0.239743,-0.559549,-0.383464,-1.133870,0.317492,1.161358,-0.907086
2,-0.297796,0.227382,-0.828725,-0.930263,-1.331941,0.000581,1.100673,-0.255681,-0.953787,0.915146,-0.184209,0.563796,-0.120268,0.006375,0.131755,0.499440,0.781800,-0.322587,-0.303924,-0.478724,-0.239859,0.768840,-0.020661,-0.875348,0.206708,-0.262722,0.563047,0.330022,0.220090,-0.651355,-0.049293,0.726039,-0.456744,0.022500,-0.840856,-0.109527,0.522686,0.346152,0.102308,0.769755,...,1.058520,0.032529,0.399757,-0.327687,-0.083737,-0.337589,0.922929,0.251694,-0.067573,1.088670,0.407071,-0.671685,0.255252,0.334191,-0.597423,-0.864936,-0.204362,-0.966903,-0.064561,0.885772,0.032412,-0.827623,0.663812,-1.438612,-0.120259,0.025848,-1.428980,-0.181340,-0.090131,-0.467942,0.931924,0.789689,0.325745,-0.162744,-0.158970,1.745260,-0.756496,-0.107627,-0.507257,0.148608
3,-0.298959,1.226538,-0.273262,-0.193495,-0.960795,-1.211861,0.651972,-0.499279,-0.519252,-0.327333,-0.771530,1.104304,-1.302379,-0.803433,0.063724,-0.327753,0.625647,-0.241537,-0.547194,0.494541,0.717901,-0.171120,0.554854,0.149232,0.115206,-0.739315,-0.096879,-1.432056,-0.094891,0.020716,-0.073888,0.206718,-0.697353,0.458862,-1.132655,-1.074679,0.898620,0.048033,0.709850,-0.172128,...,0.236200,-0.674300,0.067175,0.630835,-0.588936,-0.178568,1.164408,1.462389,0.232227,0.020143,0.678052,-0.306933,0.334124,0.088196,-0.630048,-0.839596,-0.117377,-0.439262,0.424808,0.211765,0.644426,0.288628,0.114887,-0.101396,0.558068,0.125556,0.659432,-0.713557,-0.622589,-0.580515,0.499779,0.342326,-0.274774,-0.019963,-0.176135,0.086640,-1.045700,0.795330,0.627948,-0.652297
4,0.855263,0.620701,-2.721782,-0.227145,-0.487231,0.483682,0.412447,-0.489038,-0.862797,-0.023894,0.697763,0.968274,0.751439,-0.533650,0.170204,0.051519,0.122080,0.271693,0.342446,0.051996,0.435968,0.183781,1.350144,0.574981,-0.394617,0.211196,0.502357,0.332763,0.665256,-0.431078,-1.060200,0.446521,-0.741912,-0.687671,-0.671986,-0.627480,-0.094829,0.685959,-1.053607,-0.571379,...,0.488783,-0.293612,0.442409,0.734982,0.404232,0.107475,0.472886,-0.000363,-0.219103,0.011157,0.794848,-0.487183,-1.039089,0.668677,0.307177,-1.301549,-0.449136,-1.064740,-0.753380,0.655133,0.556565,0.076706,-0.000937,-1.046833,0.257163,0.740743,-1.445098,-0.293968,-0.43691

In [ ]:
target1

0        no
1       yes
2       yes
3       yes
4       yes
       ... 
3671    yes
3672    yes
3673    yes
3674    yes
3675    yes
Name: 1, Length: 3676, dtype: object

In [ ]:
sentence_vectors2_pd2

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767
0,-0.348241,0.242336,-1.248936,-0.540791,-1.005100,-0.049697,0.912257,-0.581431,0.021589,-0.360456,-0.155772,0.917619,-0.046144,-0.411623,0.737956,0.020150,0.475779,-0.133473,-0.360397,1.034031,0.969849,-0.048550,0.304464,0.151521,-0.494420,0.844897,1.112679,-0.185123,0.773386,-1.136688,0.245787,0.603927,-0.864420,-0.110826,0.145871,-0.493775,1.123160,0.632548,-0.151503,0.561690,...,0.432546,-0.454629,0.146862,-0.913950,0.153046,0.306682,-0.137160,0.596743,0.125047,0.253388,0.486710,-0.143693,-0.089314,0.363203,-0.317778,0.083129,-0.112748,-0.328064,-0.241017,0.104790,0.453086,1.061906,-0.501667,-0.507338,-0.915340,0.179671,-1.129732,-0.802240,0.210502,-0.261477,-0.065303,0.817948,0.208688,-0.331611,0.318574,0.457819,-1.618667,0.515554,-0.029480,-0.069541
1,0.738840,-0.083606,-0.169157,0.189716,-0.665054,1.123038,0.993761,-0.031485,-1.111270,-0.458566,0.122261,1.247611,-0.001993,0.125481,0.207528,1.272199,0.269138,-0.697519,0.961733,-0.655226,0.098787,-0.174520,1.249323,-0.341930,-0.400994,-0.472511,1.075040,-1.391647,0.204980,-0.590474,-0.123559,-0.067508,-0.567652,-0.839325,-1.257739,-0.478759,-0.310350,0.435989,0.010085,0.413739,...,0.265298,0.450690,0.639474,-1.050234,0.546487,-0.106463,0.566667,0.635022,0.065892,0.688403,0.308302,-1.230280,-0.432517,-0.428875,-0.387714,-0.484045,0.070346,-1.270246,-0.394813,1.732016,-0.639712,0.103300,0.970311,-0.368027,-0.909189,0.851012,-0.077095,0.322993,-0.025627,-0.500145,0.927787,-0.737570,0.517000,0.567827,-1.076908,0.562315,-0.325158,-0.754445,-0.269006,-0.476915
2,-0.543551,0.160737,-1.555031,0.312328,0.642034,0.091409,0.334879,-0.250084,-0.277108,0.063203,-1.315711,0.269360,-0.537438,-0.091387,-0.182068,-0.589234,-0.210178,0.329153,-1.674379,-0.391463,0.183244,-0.945986,0.278354,0.131423,-0.730929,0.676797,0.459396,-0.326997,0.789579,-1.109302,-0.303607,0.427725,0.162004,-0.908431,-0.175425,-0.545184,0.323171,-0.386340,0.149102,-0.468111,...,-0.448661,-1.067641,0.252713,-0.235789,0.618081,0.180997,-0.418662,0.286374,0.656804,0.406052,0.480478,-0.543335,-0.558810,0.848189,-0.341937,-0.224254,-0.174960,0.631242,-1.295099,0.222688,-0.039473,0.692916,-0.262332,-0.537584,-0.537315,-0.674470,0.513878,-1.129909,0.330546,-0.263849,0.165664,0.812616,1.239299,-0.139820,0.011890,0.717333,-1.359864,1.249477,-0.157263,-0.650541
3,-0.387758,-0.261820,-0.793002,0.235058,0.473294,0.334991,1.369788,0.073379,0.250318,-0.193932,-0.923498,0.031503,-0.076368,-0.179714,1.069900,0.440740,0.370337,0.344834,0.343435,-0.372704,0.369832,-0.599715,1.182858,0.122570,-1.728693,0.133061,1.365694,0.251935,-0.175891,-0.671309,-0.834285,0.592314,-0.464386,0.659287,-0.038624,-1.487602,0.091686,-0.276758,-0.043035,0.108599,...,-0.472974,0.628091,0.477393,-0.735508,0.998477,-0.231949,-0.322409,0.772011,-1.070289,0.724123,0.202001,0.548036,-0.385430,-1.401007,-0.901329,0.279431,-1.005499,-0.564172,0.042216,1.202258,-0.887804,-0.512621,-0.065371,-0.097174,-0.159323,0.058370,-0.590985,0.126995,1.146165,0.502949,0.333709,0.244305,-0.757521,1.122793,-0.402754,0.876106,-1.014879,-0.855955,-0.320091,-0.429329
4,0.142582,-0.082913,-1.852174,-0.409499,-0.459822,0.640936,0.693709,-0.472426,-0.876044,0.268266,-0.564691,0.850871,0.714988,-0.666652,-0.076488,1.183062,0.760080,0.522387,-0.345889,0.223768,-0.341539,0.155611,0.982431,0.002373,-0.878573,-0.181539,-0.475759,-0.524280,1.228240,-0.515826,-1.584340,0.697412,-0.030068,0.300873,-0.631364,-0.106617,0.860223,-0.058580,-0.680088,0.160734,...,0.996334,0.720033,0.007737,-0.636855,0.584486,-0.329953,0.645096,-0.321156,-0.222426,0.482864,1.682442,-0.371170,-0.289482,1.347612,-0.437629,-0.385356,-1.553506,-0.456417,-0.349924,0.267811,-0.510566,-0.852295,0.164311,-1.834163,-0.275083,-0.031090,-0.553869,0.02428

In [ ]:
target2

0      yes
1      yes
2      yes
3      yes
4      yes
      ... 
395    yes
396    yes
397    yes
398     no
399    yes
Name: 1, Length: 400, dtype: object

In [ ]:
from sklearn.svm import SVC

c_list = [1, 10, 100,1000]
gamma_list = [0.1, 0.01,0.001]
degree_list = [2,3,4]
ker = ["linear","rbf","poly","sigmoid"]
best_score = 0
best_parameters = {}
for k in ker:
    for gamma in gamma_list:
        for C in c_list:
            for degree in degree_list:
                svm = SVC(kernel=k,degree=degree, gamma=gamma, C=C)
                svm.fit(sentence_vectors1_pd1, target1)
                score = svm.score(sentence_vectors2_pd2, target2)
                # 最も良いスコアのパラメータとスコアを更新
                if score > best_score:
                    best_score = score
                    best_parameters = {'kernel' : k, 'degree' : degree, 'gamma' : gamma, 'C' : C}

print('Best score: {}'.format(best_score))
print('Best parameters: {}'.format(best_parameters))


Best score: 0.9225
Best parameters: {'kernel': 'poly', 'degree': 4, 'gamma': 0.1, 'C': 1}


In [ ]:
from sklearn.svm import SVC
from sklearn import svm
from sklearn.metrics import classification_report

classifier = svm.SVC(kernel="poly", C=1,degree = 4, gamma=0.1)
classifier.fit(sentence_vectors1_pd1,target1)

# 予測
predict_label2 = classifier.predict(sentence_vectors2_pd2)
test_label = target2
target_names = ["yes", "no"]
result2 = classification_report(test_label, predict_label2,target_names=target_names)

print(result2)

              precision    recall  f1-score   support

         yes       0.94      0.71      0.81        93
          no       0.92      0.99      0.95       307

    accuracy                           0.92       400
   macro avg       0.93      0.85      0.88       400
weighted avg       0.92      0.92      0.92       400



In [ ]:
from sklearn import datasets
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC

In [ ]:
tuned_parameters = [
    {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
    {'C': [1, 10, 100, 1000], 'kernel': ['rbf'], 'gamma': [0.1, 0.01,0.001]},
    {'C': [1, 10, 100, 1000], 'kernel': ['poly'], 'degree': [2, 3, 4], 'gamma': [0.1,0.01, 0.001]},
    {'C': [1, 10, 100, 1000], 'kernel': ['sigmoid'], 'gamma': [0.1, 0.01,0.001]}
    ]

In [ ]:
score = 'accuracy'
clf = GridSearchCV(
    SVC(), 
    tuned_parameters, 
    cv=5
    ) 

In [ ]:
clf.fit(sentence_vectors1_pd1,target1)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid=[{'C': [1, 10, 100, 1000], 'kernel': ['linear']},
                         {'C': [1, 10, 100, 1000], 'gamma': [0.1, 0.01, 0.001],
                          'kernel': ['rbf']},
                         {'C': [1, 10, 100, 1000], 'degree': [2, 3, 4],
                          'gamma': [0.1, 0.01, 0.001], 'kernel': ['poly']},
                         {'C': [1, 10, 100, 1000], 'gamma': [0.1, 0.01, 0.001],
                          'kernel': ['sigmoid']}])

In [ ]:
clf.best_params_

{'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}

In [ ]:
y_true,y_pred = target2,clf.predict(sentence_vectors2_pd2)
print(classification_report(y_true,y_pred))

              precision    recall  f1-score   support

          no       0.86      0.73      0.79        93
         yes       0.92      0.96      0.94       307

    accuracy                           0.91       400
   macro avg       0.89      0.85      0.87       400
weighted avg       0.91      0.91      0.91       400



In [ ]:
from sklearn.svm import SVC
from sklearn import svm
from sklearn.metrics import classification_report

classifier = svm.SVC(kernel="rbf", C=10, gamma=0.001)
classifier.fit(sentence_vectors1_pd1,target1)

# 予測
predict_label2 = classifier.predict(sentence_vectors2_pd2)
test_label = target2
target_names = ["yes", "no"]
result2 = classification_report(test_label, predict_label2,target_names=target_names)

print(result2)

              precision    recall  f1-score   support

         yes       0.86      0.73      0.79        93
          no       0.92      0.96      0.94       307

    accuracy                           0.91       400
   macro avg       0.89      0.85      0.87       400
weighted avg       0.91      0.91      0.91       400



In [ ]:
bertgrid_pd = pd.Series(predict_label2)
print(bertgrid_pd)

0      yes
1      yes
2      yes
3      yes
4      yes
      ... 
395    yes
396    yes
397    yes
398     no
399    yes
Length: 400, dtype: object


In [ ]:
comp_grid_bert = bertgrid_pd.compare(target2)
comp_grid_bert

,self,other
18,yes,no
37,no,yes
45,yes,no
60,no,yes
73,yes,no
81,yes,no
98,yes,no
136,yes,no
155,yes,no
160,no,yes


In [ ]:
l_index1 = list(comp_grid_bert.index)
print(l_index1)

[18, 37, 45, 60, 73, 81, 98, 136, 155, 160, 163, 168, 175, 180, 195, 196, 198, 208, 223, 233, 237, 256, 258, 260, 265, 273, 288, 291, 296, 324, 338, 348, 362, 368, 371, 386]


In [ ]:
false_no_result = []
false_yes_result = []

for i in l_index1:
    number = i
    false_sentense = explain2[i]
    correct_label1 = target2[i]
    if correct_label1 == "no":
        false_no_result.append(false_sentense)
    elif correct_label1 == "yes":
        false_yes_result.append(false_sentense)
    # false_result.append(false_sentense)




In [ ]:
false_no_result_pd = pd.Series(false_no_result)
print(false_no_result_pd)

0                                               泉質が単純泉？
1                                     7日間のウィークリープランでした。
2                                     でもま、安くてそこまでは甘いかな？
3                               多忙期の疲れをとる為に、息抜きで利用しました。
4                                       細かい話で申し訳ございません。
5                 ３０分後ぐらい部屋に電話をくれ「まだ、音は響きますか？」と連絡くれました。
6                                優良ホテルはへたを取っているケース多いです。
7                                         雨に濡れないのも重要です。
8                          自社ＨＰでは会員料を取っているが、楽天の方が設定も安い。
9         繁忙期だったためか、都内のホテルの選択肢が少なく、初めてチェックインすることになりました。
10    アウトリガーに楽天トラベルのオフィスがあることから、このアウトリガーグループのホテルは楽天が...
11                            他の方の評価がよかったので初めて利用してみました。
12                                       宿泊料金の安さで選びました。
13                           近くに競争相手がいないからこそ出来る事だと思います。
14                                   電子レンジの設置と空気清浄機の貸出。
15                                       川を境に泉質が違うようです。
16                          更にポイントが使えたので２人で１万円という価格でした。
17                                本当にお世話になりありがとう

In [ ]:
false_yes_result_pd = pd.Series(false_yes_result)
print(false_yes_result_pd)

0                               家族みんなでごはんをおかわりしてしまいました。
1     予約よりも高く取られていたのか、Tx込みでの合計額なのか分からず、帰国後の私の旅行支出チェッ...
2                    朝はやはりパンとハムエッグとコーヒーの3点セットが食べたかったです。
3                                           時間によって混みます。
4                                 空港直結なところに惹かれて、宿泊しました。
5                               東京に宿泊の際は、こちらのホテルに決定です！！
6                                       ぜひまた利用したいホテルです。
7                                               関心しました。
8                                  太田駅前の立地で安かったので決めました。
9                     うちの両親もとても気に入っていたので、ぜひまた両親と行きたいです。
10                                            これはスゴイです。
dtype: object


bag of words と　Sentence BERTモデルの正誤の一致度を測定


In [ ]:
l1_l2_and = set(l_index1) & set(l_index2)
print(l1_l2_and)

print(type(l1_l2_and))


In [ ]:
# Bag of Wordsモデルに於ける誤り数
print(len(l_index1))
# Sentence Bertモデルに於ける誤り数
print(len(l_index2))
# 二つのモデルの共通する誤り数
print(len(l1_l2_and))
# Bag of Wordsモデルの誤判定数に於ける両モデル共通の誤り数の割合
falserate = len(l1_l2_and)/len(l_index1)
print(falserate)

**共通して誤っている説明文を抽出**

In [ ]:
false_result3 = []

for i in l1_l2_and:
    number = i
    false_sentense3 = explain2[i]
    correct_label2 = target2[i]
    false_result3.append(correct_label2 + " " + str(number) + " " +  false_sentense3)
    # false_result.append(false_sentense)

In [ ]:
false_result3

**Google Colab 表示の省略をすべて表示する方法**


In [ ]:
# pd.set_option('display.max_rows', None)
print(explain2)